In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('Online_Retail.csv')
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/01/2010 08:34,1.69,13047.0,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/01/2010 08:34,5.95,13047.0,United Kingdom


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54214 entries, 0 to 54213
Data columns (total 8 columns):
InvoiceNo      54214 non-null object
StockCode      54214 non-null object
Description    54080 non-null object
Quantity       54214 non-null int64
InvoiceDate    54214 non-null object
UnitPrice      54214 non-null float64
CustomerID     40643 non-null float64
Country        54214 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 3.3+ MB


В файле 54214 записей. Есть пропущенные значения.   
По столбцам:   
- **InvoiceNo** - номер заказа. Пропущенных значений нет.  
- **StockCode** - скорее всего, это номер товара на складе (потому что кол-во уникальных значений в этом столбце примерно соответствует кол-ву уникальных значений в столбце Description). Пропущенных значений тоже нет. Но непонятно, какие значимые выводы можно сделать на основе этих данных в рамках настоящего исследования. Проще всего этот столбец удалить для экономии места.   
- **Description** - описание товара. Есть пропущенные значения, но их немного. Для этих строк есть номер заказа, но нет стоимости товара и кол-во местами имеет отрицательное значение. Возможно, это какой-то сбой системы, или, например, заказанного товара по факту не оказалось на складе или это подарки к заказу - в любом случае, на этом мы не зарабатываем. Чтобы не гадать - проще сразу удалять.   
- **Quantity** - кол-во позиций в заказе. Пропущенных значений нет, но есть отрицательные значения в количестве 1083 строки. Будем считать, что это заказы/позиции в заказе, отмененные по какой-либо причине (потерянные, поврежденные, неправильно заказанные и т.д.). В любом случае, прибыли по ним не было. Теоретически, можно будет посмотреть на сумму недополученной прибыли, насколько она велика в масштабах всего бизнеса.
- **InvoiceDate** - дата и время заказа. Пропущенных значений нет, из этих данных лучше только дату заказа. Вряд ли время имеет смысл анализировать в данном конкретном случае.
- **UnitPrice** - стоимость единицы товара. пропущенных значений нет, но есть нулевые значения.  
- **CustomerID** - ID покупателя. Порядка 20 % не заполнено. Возможно, это покупатели, которые делали заказ без регистрации на сайте.
- **Country** - страна доставки заказа. Пропущенных значений нет.   
    
    Теперь можно подготовить данные для дальнейшей работы

In [3]:
# кол-во уникальных значений в столбце
data['StockCode'].nunique()

3351

In [4]:
# кол-во уникальных значений в столбце
data['Description'].nunique()

3401

In [5]:
# форматируем время
from datetime import datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format = '%m/%d/%Y %H:%M')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047.0,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:02:00,2.75,17850.0,United Kingdom


In [6]:
# заменяем все пропуски на nan
data.replace('', np.nan, inplace=True)

In [7]:
# В столбце CustomerID заменяем nan на неавторизованных пользователей
data['CustomerID'] = data['CustomerID'].fillna('Unauthorized_users')

In [8]:
# Создаем столбец с делением пользователей на зарегистрированных и незарегистрированных
data.loc[(data.CustomerID == 'Unauthorized_users'), 'Users'] = 'Unauthorized'
data.loc[(data.CustomerID != 'Unauthorized_users'), 'Users'] = 'Authorized'
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Users
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Authorized
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,Authorized
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047,United Kingdom,Authorized


Теперь разберемся со столбцом Quantity. Нулевых значений нет. Есть 1083 отрицательных значения. Выгрузим их в отдельный датафрейм и посмотрим, что туда попало. Среди перечислений наименований товаров встречаются: '?', '??', 'sold as set by dotcom', 'wrong barcode (22467)', "thrown away-can't sell.", 'wet/rusty', 'damaged',  'historic computer difference?....se',  'lost in space', 'mixed up', 'missing'. Видимо, это все те заказанные товары, которые по какой-либо причине не доехали до покупателей, либо были отменены/возвращены. Они составляют примерно 2 % от всех заказанных наименований (1083 / 54214 = 0,02). Их стоимость равна нулю и их проще просто удалить, потому что имеющейся информации недостаточно для их подробного анализа. У нас нет:  
- унифицированных данных о причинах отмены/возврата заказа
- бессмысленно считать кол-во строк или общее кол-во таких товаров, без возможности оценить их стоимость (нашу недополученную прибыль)
- 2 % от общего кол-ва строк - это немного, этими данными можно пренебречь, поскольку неясно, как их однозначно интерпретировать. 


In [9]:
# смотрим есть ли кол-во = 0. Такого у нас нет. 
data.loc[(data.Quantity == 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Users


In [10]:
# Датафрейм со строками с минусовым кол-вом заказанных товаров.
data_10 = data.loc[(data.Quantity < 0)]
data_10

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Users
19,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom,Authorized
30,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom,Authorized
31,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom,Authorized
202,C536548,20957,PORCELAIN HANGING BELL SMALL,-1,2010-12-01 14:33:00,1.45,12472,Germany,Authorized
375,C536642,21463,MIRRORED DISCO BALL,-1,2010-12-02 11:56:00,5.95,14390,United Kingdom,Authorized
...,...,...,...,...,...,...,...,...,...
53704,C581229,79191C,RETRO PLASTIC ELEPHANT TRAY,-24,2011-12-08 10:14:00,0.85,12558,USA,Authorized
53705,C581229,21507,ELEPHANT BIRTHDAY CARD,-12,2011-12-08 10:14:00,0.42,12558,USA,Authorized
53765,C581322,22666,RECIPE BOX PANTRY YELLOW DESIGN,-1,2011-12-08 11:51:00,2.95,18223,United Kingdom,Authorized
53770,C581330,22959,WRAP CHRISTMAS VILLAGE,-25,2011-12-08 11:57:00,0.42,15877,United Kingdom,Authorized


In [11]:
# Описание товаров с отрицательным количеством. Вывод убрала, т.к. слишком длинный
# data_10['Description'].unique()

In [12]:
# Удаляем строки с кол-вом меньше 0
data = data.loc[(data.Quantity > 0)]
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Users
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Authorized
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,Authorized
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047,United Kingdom,Authorized


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53131 entries, 0 to 54213
Data columns (total 9 columns):
InvoiceNo      53131 non-null object
StockCode      53131 non-null object
Description    53081 non-null object
Quantity       53131 non-null int64
InvoiceDate    53131 non-null datetime64[ns]
UnitPrice      53131 non-null float64
CustomerID     53131 non-null object
Country        53131 non-null object
Users          53131 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 4.1+ MB


Смотрим информацию по обновленному датафрейму и видим, что у нас остались пропущенные значения в столбце с описаниями товаров. Выведем их и обнаружим, что у всех таких позиций стоимость равна нулю. И всего их 50 штук, что совсем немного и их тоже проще удалить, чем гадать, что это за товары с нулевой стоимостью.

In [14]:
data_1 = data.loc[pd.isnull(data.Description),:]
data_1.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Users
204,536553,37461,NaN,3,2010-12-01 14:35:00,0.0,Unauthorized_users,United Kingdom,Unauthorized
730,536995,35951,NaN,57,2010-12-03 15:29:00,0.0,Unauthorized_users,United Kingdom,Unauthorized
1925,537876,84977,NaN,2,2010-12-08 18:08:00,0.0,Unauthorized_users,United Kingdom,Unauthorized


In [15]:
data_1['UnitPrice'].unique()

array([0.])

In [16]:
data = data.dropna(subset = ['Description'])
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Users
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Authorized
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,Authorized
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047,United Kingdom,Authorized


И теперь для экономии места удалим столбец:   
- StockCode - т.к. непонятно, как использовать эту информацию  

И добавим столбец с общей стоимостью заказанных товаров

In [17]:
data_fin = data.drop(['StockCode'], axis = 1)
data_fin.head(3)

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Users
0,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Authorized
1,536367,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,Authorized
2,536367,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047,United Kingdom,Authorized


In [18]:
data_fin['Cost'] = data_fin['Quantity']*data_fin['UnitPrice']
data_fin.head(3)

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Users,Cost
0,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Authorized,20.34
1,536367,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,Authorized,54.08
2,536367,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047,United Kingdom,Authorized,17.85


In [19]:
# изменим порядок столбцов
new_dataframe = data_fin.reindex(columns=['InvoiceDate', 'CustomerID', 'Users', 'InvoiceNo', 'Description', 'Quantity', 'UnitPrice', 'Cost', 'Country'])
new_dataframe.head(3)

,InvoiceDate,CustomerID,Users,InvoiceNo,Description,Quantity,UnitPrice,Cost,Country
0,2010-12-01 08:26:00,17850,Authorized,536365,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34,United Kingdom
1,2010-12-01 08:34:00,13047,Authorized,536367,ASSORTED COLOUR BIRD ORNAMENT,32,1.69,54.08,United Kingdom
2,2010-12-01 08:34:00,13047,Authorized,536367,LOVE BUILDING BLOCK WORD,3,5.95,17.85,United Kingdom


In [24]:
# new_dataframe.to_csv('data_retail.csv', index=False)

In [29]:
# группируем стоимость и кол-во товаров по заказам
data_gr = new_dataframe[['InvoiceNo', 'Quantity', 'Cost']].groupby('InvoiceNo').sum().reset_index()
data_gr

,InvoiceNo,Quantity,Cost
0,536365,6,20.34
1,536367,35,71.93
2,536368,3,14.85
3,536373,8,22.00
4,536375,6,15.30
...,...,...,...
13953,581579,26,66.90
13954,581580,4,22.63
13955,581581,6,17.34
13956,581585,73,60.54


In [38]:
# Переименовываем столбцы, чтобы не спутать с предыдущими данными
data_gr.columns = ['InvoiceNo', 'Quantity_by_order', 'Cost_by_order']
data_gr

,InvoiceNo,Quantity_by_order,Cost_by_order
0,536365,6,20.34
1,536367,35,71.93
2,536368,3,14.85
3,536373,8,22.00
4,536375,6,15.30
...,...,...,...
13953,581579,26,66.90
13954,581580,4,22.63
13955,581581,6,17.34
13956,581585,73,60.54


In [46]:
# Делаем датафрейм ключ со значениями всех остальных параметров для заказа
data_info = new_dataframe.drop_duplicates(subset=['InvoiceNo'], keep = 'first' )
data_info

,InvoiceDate,CustomerID,Users,InvoiceNo,Description,Quantity,UnitPrice,Cost,Country
0,2010-12-01 08:26:00,17850,Authorized,536365,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34,United Kingdom
1,2010-12-01 08:34:00,13047,Authorized,536367,ASSORTED COLOUR BIRD ORNAMENT,32,1.69,54.08,United Kingdom
3,2010-12-01 08:34:00,13047,Authorized,536368,BLUE COAT RACK PARIS FASHION,3,4.95,14.85,United Kingdom
4,2010-12-01 09:02:00,17850,Authorized,536373,CREAM CUPID HEARTS COAT HANGER,8,2.75,22.00,United Kingdom
5,2010-12-01 09:32:00,17850,Authorized,536375,WOODEN FRAME ANTIQUE WHITE,6,2.55,15.30,United Kingdom
...,...,...,...,...,...,...,...,...,...
54201,2011-12-09 12:19:00,17581,Authorized,581579,JUMBO STORAGE BAG SUKI,10,1.79,17.90,United Kingdom
54204,2011-12-09 12:20:00,12748,Authorized,581580,12 PENCILS SMALL TUBE SKULL,1,0.65,0.65,United Kingdom
54207,2011-12-09 12:20:00,17581,Authorized,581581,SET OF 6 RIBBONS PARTY,6,2.89,17.34,United Kingdom
54208,2011-12-09 12:31:00,15804,Authorized,581585,BLACK TEA TOWEL CLASSIC DESIGN,12,0.39,4.68,United Kingdom


In [47]:
#Объединяем дф с параметрами заказов и дф с прочими данными
data_grouped = data_gr.merge(data_info, on = ['InvoiceNo'], how = 'left')
data_grouped

,InvoiceNo,Quantity_by_order,Cost_by_order,InvoiceDate,CustomerID,Users,Description,Quantity,UnitPrice,Cost,Country
0,536365,6,20.34,2010-12-01 08:26:00,17850,Authorized,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34,United Kingdom
1,536367,35,71.93,2010-12-01 08:34:00,13047,Authorized,ASSORTED COLOUR BIRD ORNAMENT,32,1.69,54.08,United Kingdom
2,536368,3,14.85,2010-12-01 08:34:00,13047,Authorized,BLUE COAT RACK PARIS FASHION,3,4.95,14.85,United Kingdom
3,536373,8,22.00,2010-12-01 09:02:00,17850,Authorized,CREAM CUPID HEARTS COAT HANGER,8,2.75,22.00,United Kingdom
4,536375,6,15.30,2010-12-01 09:32:00,17850,Authorized,WOODEN FRAME ANTIQUE WHITE,6,2.55,15.30,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...
13953,581579,26,66.90,2011-12-09 12:19:00,17581,Authorized,JUMBO STORAGE BAG SUKI,10,1.79,17.90,United Kingdom
13954,581580,4,22.63,2011-12-09 12:20:00,12748,Authorized,12 PENCILS SMALL TUBE SKULL,1,0.65,0.65,United Kingdom
13955,581581,6,17.34,2011-12-09 12:20:00,17581,Authorized,SET OF 6 RIBBONS PARTY,6,2.89,17.34,United Kingdom
13956,581585,73,60.54,2011-12-09 12:31:00,15804,Authorized,BLACK TEA TOWEL CLASSIC DESIGN,12,0.39,4.68,United Kingdom


In [49]:
# Убираем лишние столбцы
data_grouped = data_grouped.drop(['Description', 'Quantity', 'UnitPrice', 'Cost'], axis = 1)
data_grouped.head(3)

,InvoiceNo,Quantity_by_order,Cost_by_order,InvoiceDate,CustomerID,Users,Country
0,536365,6,20.34,2010-12-01 08:26:00,17850,Authorized,United Kingdom
1,536367,35,71.93,2010-12-01 08:34:00,13047,Authorized,United Kingdom
2,536368,3,14.85,2010-12-01 08:34:00,13047,Authorized,United Kingdom


In [50]:
data_grouped.columns = ['order_ID', 'goods_quantity', 'total_cost', 'date', 'customer_ID', 'customers', 'country']
data_grouped

,order_ID,goods_quantity,total_cost,date,customer_ID,customers,country
0,536365,6,20.34,2010-12-01 08:26:00,17850,Authorized,United Kingdom
1,536367,35,71.93,2010-12-01 08:34:00,13047,Authorized,United Kingdom
2,536368,3,14.85,2010-12-01 08:34:00,13047,Authorized,United Kingdom
3,536373,8,22.00,2010-12-01 09:02:00,17850,Authorized,United Kingdom
4,536375,6,15.30,2010-12-01 09:32:00,17850,Authorized,United Kingdom
...,...,...,...,...,...,...,...
13953,581579,26,66.90,2011-12-09 12:19:00,17581,Authorized,United Kingdom
13954,581580,4,22.63,2011-12-09 12:20:00,12748,Authorized,United Kingdom
13955,581581,6,17.34,2011-12-09 12:20:00,17581,Authorized,United Kingdom
13956,581585,73,60.54,2011-12-09 12:31:00,15804,Authorized,United Kingdom


In [51]:
data_grouped.to_csv('data_by_orders.csv', index=False)

In [52]:
data_grouped['customer_ID'].nunique()

3793